# CRÉATION DE MODÈLES PRÉDICTIFS

In [86]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

df = pd.read_csv("winequality-red.csv")

## Régréssion Linéaire

In [87]:
X = df.drop(['quality'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, df["quality"], test_size=0.2, random_state=42)

### Modèle 1 - Basique

In [88]:
model = LinearRegression().fit(X_train, y_train)
model

LinearRegression()

In [89]:
model.score(X_test, y_test)

0.40318034127962266

### Modèle 2 - MinMaxScaler

In [90]:
# On cree un pipeline de proprocessing pour les variables numériques
numeric_features = X.select_dtypes(include=['float', 'int']).columns.tolist()

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler


numeric_transformer = Pipeline([
        ('stdscaler', MinMaxScaler()),  # moyenne nulle et écart type = 1 -> Reg, SVM, PCA
        ])

from sklearn.preprocessing import OneHotEncoder
categorical_transformer = OneHotEncoder()


# a l'aide de la classe ColumnTransformer, 
# on déclare à quelles variables on applique quel transformer
from sklearn.compose import ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features)
    ]
)

#On obtient un pipeline de preprocessing qu'on peut utiliser dans un pipeline d'entainement
linear = LinearRegression()

pipe = Pipeline([
     ('prep', preprocessor),
     ('linear', linear)
])

# Entraîner le modèle sur les données d'entraînement
pipe.fit(X_train, y_train)

# Évaluer la performance du modèle sur les données de test
score = pipe.score(X_test, y_test)
print(f"Score sur les données de test: {score:.3f}")

Score sur les données de test: 0.403


### Modèle 3 - Polynomiales Features

In [91]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures

# On ajoute PolynomialFeatures dans notre pipeline de prétraitement
numeric_transformer = Pipeline([
        ('poly', PolynomialFeatures(degree=2, include_bias=False)),
        ('stdscaler', MinMaxScaler())
])

# a l'aide de la classe ColumnTransformer, 
# on déclare à quelles variables on applique quel transformer
from sklearn.compose import ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
    ]
)

#On obtient un pipeline de preprocessing qu'on peut utiliser dans un pipeline d'entainement
linear = LinearRegression()

pipe = Pipeline([
     ('prep', preprocessor),
     ('linear', linear)
])

# Entraîner le modèle sur les données d'entraînement
pipe.fit(X_train, y_train)

# Évaluer la performance du modèle sur les données de test
score = pipe.score(X_test, y_test)
print(f"Score sur les données de test: {score:.3f}")

Score sur les données de test: 0.416


### Modèle 4 - Ridge

In [92]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures 
from sklearn.linear_model import Ridge

numeric_transformer = Pipeline([
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),
    ('stdscaler', MinMaxScaler())
])

# Utiliser ColumnTransformer pour appliquer le pré-traitement à chaque type de variable
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
    ]
)

# Définir un modèle de régression Ridge avec une valeur d'alpha de 1
ridge = Ridge(alpha=1)

# Créer un pipeline final qui applique le prétraitement et la régression Ridge
pipe = Pipeline([
    ('prep', preprocessor),
    ('ridge', ridge)
])

# Entraîner le modèle sur les données d'entraînement
pipe.fit(X_train, y_train)

# Évaluer la performance du modèle sur les données de test
score = pipe.score(X_test, y_test)
print(f"Score sur les données de test: {score:.3f}")

Score sur les données de test: 0.429


In [93]:
# Entrainement sur X_train
trained_pipe = pipe.fit(X_train,y_train)

# prediction sur X_test
y_pred=trained_pipe.predict(X_test)

# scoring sur X_test
score = trained_pipe.score(X_test,y_test)

print(score)
print(y_pred)
print(y_test)

0.428863698436568
[5.41541482 5.07852091 5.74173536 5.43982961 5.72214828 5.23009829
 5.09225796 5.04579926 5.78637909 5.7012053  6.23566215 5.22188441
 5.58687187 5.28959759 5.40555788 6.42345003 5.23038494 5.5615684
 6.73772261 5.43065305 5.09189172 5.21429994 5.84146804 6.30489433
 5.36134454 5.4566786  6.19272095 5.34547999 5.22921754 6.26688235
 5.26691554 5.37861249 5.77253695 5.26419393 5.52914966 5.03828927
 6.25501393 5.76283687 5.60637469 6.09386612 5.48569308 5.24774469
 6.22548518 5.27015007 6.09153057 5.98804821 6.57659727 5.5375726
 5.13305344 5.5658485  5.08433226 5.04152309 5.55516907 6.44850478
 4.9608125  5.0555761  6.10473935 5.37977139 5.80797513 5.26055214
 5.6200205  5.95251069 5.26424097 5.23982933 6.53289823 5.34238963
 6.3808879  5.3480738  6.47007946 5.27113648 6.35538692 4.87030219
 5.84242718 5.80371663 6.17808994 5.25965121 6.82554234 5.76267475
 6.0814531  6.38330464 5.2979865  6.51813984 5.47423781 5.56650699
 5.755914   6.43743109 5.28567261 5.910301   6

In [94]:
import pickle
with open('trained_pipeline.pkl', 'wb') as f:
    pickle.dump(trained_pipe, f)

### Modèle 5 : RobustScaler

In [95]:
from sklearn.preprocessing import RobustScaler

# Créer un pipeline de prétraitement pour les variables numériques avec RobustScaler
numeric_transformer = Pipeline([
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),
    ('robustscaler', RobustScaler())
])

# Utiliser ColumnTransformer pour appliquer le pré-traitement à chaque type de variable
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
    ]
)

# Créer un pipeline final qui applique le prétraitement et la régression Ridge
pipe = Pipeline([
    ('prep', preprocessor),
    ('ridge', ridge)
])

# Entraîner le modèle sur les données d'entraînement
pipe.fit(X_train, y_train)

# Évaluer la performance du modèle sur les données de test
score = pipe.score(X_test, y_test)
print(f"Score sur les données de test avec RobustScaler: {score}")


Score sur les données de test avec RobustScaler: 0.42260240184814746


In [96]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.neighbors import KNeighborsRegressor


# Créer un pipeline de prétraitement pour les variables numériques
numeric_features = X.select_dtypes(include=['float', 'int']).columns.tolist()
numeric_transformer = Pipeline([
    ('robustscaler', RobustScaler())
])


# Utiliser ColumnTransformer pour appliquer le pré-traitement à chaque type de variable
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
    ]
)

# Définir un modèle de régression Ridge avec une valeur d'alpha de 1
knn = KNeighborsRegressor()

# Créer un pipeline final qui applique le prétraitement et la régression Ridge
pipe = Pipeline([
    ('prep', preprocessor),
    ('knn', knn)
])

# Définir les paramètres à tester
param_grid = {
    'knn__n_neighbors': range(1, 21),
    'knn__weights': ['uniform', 'distance']
}

# Initialiser le GridSearchCV
grid_search = GridSearchCV(pipe, param_grid=param_grid, cv=5)

# Adapter le GridSearchCV aux données d'entraînement
grid_search.fit(X_train, y_train)

# Afficher les meilleurs hyperparamètres et le score correspondant
print("Best parameters:", grid_search.best_params_)
print("Best score:", grid_search.best_score_)

# Utiliser les meilleurs hyperparamètres pour entraîner un nouveau modèle
best_params = grid_search.best_params_
knn_best = KNeighborsRegressor(n_neighbors=best_params['knn__n_neighbors'], 
                                weights=best_params['knn__weights'])
pipe_best = Pipeline([
     ('prep', preprocessor),
     ('knn', knn_best)
])

# Entraîner le modèle sur les données d'entraînement
pipe_best.fit(X_train, y_train)

# Évaluer la performance du modèle sur les données de test
score = pipe_best.score(X_test, y_test)
print(f"Score sur les données de test: {score:.3f}")

Best parameters: {'knn__n_neighbors': 20, 'knn__weights': 'distance'}
Best score: 0.41439067853705563
Score sur les données de test: 0.490
